In [1]:
import openml
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder #For encoding categorical variables
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans,AgglomerativeClustering
from sklearn.metrics import accuracy_score,f1_score,adjusted_rand_score,silhouette_score
from joblib import Parallel,delayed
import time

# Download Datasets Using openml

In [2]:
iris = openml.datasets.get_dataset("iris")
iris_df, iris_label, categorical_indicator, attribute_names = iris.get_data(
    target=iris.default_target_attribute, dataset_format="dataframe"
)
iris_df["class"]=iris_label
iris_x=iris_df.iloc[:,:4]
iris_df

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [3]:
wine = openml.datasets.get_dataset("wine")
wine_df, wine_label, categorical_indicator, attribute_names = wine.get_data(
    target= wine.default_target_attribute, dataset_format="dataframe"
)
wine_df["class"]=wine_label
wine_x=wine_df.iloc[:,:13]
wine_df

,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280%2FOD315_of_diluted_wines,Proline,class
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,3
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,3
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,3
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,3


# Identify Data Types

In [4]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   sepallength  150 non-null    float64 
 1   sepalwidth   150 non-null    float64 
 2   petallength  150 non-null    float64 
 3   petalwidth   150 non-null    float64 
 4   class        150 non-null    category
dtypes: category(1), float64(4)
memory usage: 5.1 KB


In [5]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   Alcohol                         178 non-null    float64 
 1   Malic_acid                      178 non-null    float64 
 2   Ash                             178 non-null    float64 
 3   Alcalinity_of_ash               178 non-null    float64 
 4   Magnesium                       178 non-null    uint8   
 5   Total_phenols                   178 non-null    float64 
 6   Flavanoids                      178 non-null    float64 
 7   Nonflavanoid_phenols            178 non-null    float64 
 8   Proanthocyanins                 178 non-null    float64 
 9   Color_intensity                 178 non-null    float64 
 10  Hue                             178 non-null    float64 
 11  OD280%2FOD315_of_diluted_wines  178 non-null    float64 
 12  Proline               

# Transform categorical variable to numeric

In [6]:
le=LabelEncoder()

iris_y=le.fit_transform(iris_label)
wine_y=le.fit_transform(wine_label)

# Min-max normalise

In [7]:
scaler = MinMaxScaler()

iris_x_scaled=scaler.fit_transform(iris_x)
wine_x_scaled=scaler.fit_transform(wine_x)


# DataSets

In [8]:
Dataset = ["Iris","Wine"]

# KMeans

In [9]:
def kmeans (x, y, parameters):
    start_time = time.time()
    kmeans = KMeans(n_clusters = parameters[0], max_iter = parameters[1], n_init = parameters[2])
    y_kmeans = kmeans.fit_predict(x)
    kmeans_f1score = f1_score(y, y_kmeans, average = 'weighted')
    kmeans_ars = adjusted_rand_score(y, y_kmeans)
    kmeans_sscore = silhouette_score(x, y_kmeans, metric='euclidean')
    kmeans_execution_time = time.time() - start_time
    return y_kmeans, kmeans_f1score, kmeans_ars, kmeans_sscore, kmeans_execution_time

In [10]:
n_clusters=[*[2,3,4],*[5,6,7]]
max_iter=[200,300,400]
n_init=[5,10,15]
    
kmean_parameters = pd.DataFrame({"n_clusters":[] , 
                                 "max_iter":[] , 
                                 "n_init":[]}).astype(int)


In [11]:
for i in n_clusters:
        for  ite in max_iter:
            for n in n_init:
                kmean_parameters = kmean_parameters.append(
                    {'n_clusters' : i, 'max_iter' : ite, 'n_init' : n},ignore_index=True
                )

In [12]:
kmean_parameters

,n_clusters,max_iter,n_init
0,2,200,5
1,2,200,10
2,2,200,15
3,2,300,5
4,2,300,10
5,2,300,15
6,2,400,5
7,2,400,10
8,2,400,15
9,3,200,5


In [13]:
final_iris_kmeans = Parallel(n_jobs=-1)(delayed(kmeans)(iris_x_scaled, iris_y, kmean_parameters.iloc[i]) for i in range(0, len(kmean_parameters)))
final_wine_kmeans = Parallel(n_jobs=-1)(delayed(kmeans)(wine_x_scaled, wine_y, kmean_parameters.iloc[i]) for i in range(0, len(kmean_parameters)))

In [14]:
final_kmeans_df=pd.DataFrame({
    "Dataset":[],
    "n_clusters":[],
    "max_iter":[],
    "n_init":[],
    "f1 score":[],
    "Adjusted Random Score":[],
    "Silhouette Score":[],
    "Execution Time":[],
})

In [15]:
for i in range(0, len(kmean_parameters)):
    final_kmeans_df =final_kmeans_df.append({
        "Dataset":Dataset[0],
        "n_clusters":kmean_parameters.iloc[i][0],
        "max_iter":kmean_parameters.iloc[i][1],
        "n_init":kmean_parameters.iloc[i][2],
        "f1 score":final_iris_kmeans[i][1],
        "Adjusted Random Score":final_iris_kmeans[i][2],
        "Silhouette Score":final_iris_kmeans[i][3],
        "Execution Time":final_iris_kmeans[i][4],
        "Dataset":Dataset[0]
        },ignore_index=True)
    
for i in range(0, len(kmean_parameters)):
    final_kmeans_df =final_kmeans_df.append({
        "Dataset":Dataset[0],
        "n_clusters":kmean_parameters.iloc[i][0],
        "max_iter":kmean_parameters.iloc[i][1],
        "n_init":kmean_parameters.iloc[i][2],
        "f1 score":final_wine_kmeans[i][1],
        "Adjusted Random Score":final_wine_kmeans[i][2],
        "Silhouette Score":final_wine_kmeans[i][3],
        "Execution Time":final_wine_kmeans[i][4],
        "Dataset":Dataset[1]
        },ignore_index=True)
final_kmeans_df

,Dataset,n_clusters,max_iter,n_init,f1 score,Adjusted Random Score,Silhouette Score,Execution Time
0,Iris,2.0,200.0,5.0,0.000000,0.568116,0.629468,0.015621
1,Iris,2.0,200.0,10.0,0.000000,0.568116,0.629468,0.015623
2,Iris,2.0,200.0,15.0,0.000000,0.568116,0.629468,0.046864
3,Iris,2.0,300.0,5.0,0.000000,0.568116,0.629468,0.031244
4,Iris,2.0,300.0,10.0,0.000000,0.568116,0.629468,0.031243
...,...,...,...,...,...,...,...,...
103,Wine,7.0,300.0,10.0,0.734817,0.595347,0.212950,0.062485
104,Wine,7.0,300.0,15.0,0.000000,0.581690,0.205585,0.078111
105,Wine,7.0,400.0,5.0,0.387666,0.487093,0.134210,0.031243
106,Wine,7.0,400.0,10.0,0.169768,0.561410,0.215618,0.046869


# Agglomerative Clustering

In [ ]:
# def agglomerative (x, y, parameters):
#     start_time = time.time()
#     agglomerative= KMeans(n_clusters = parameters[0], max_iter = parameters[1], n_init = parameters[2])
#     y_kmeans = kmeans.fit_predict(x)
#     kmeans_f1score = f1_score(y, y_kmeans, average = 'weighted')
#     kmeans_ars = adjusted_rand_score(y, y_kmeans)
#     kmeans_sscore = silhouette_score(x, y_kmeans, metric='euclidean')
#     kmeans_execution_time = time.time() - start_time
#     return y_kmeans, kmeans_f1score, kmeans_ars, kmeans_sscore, kmeans_execution_time

# DBScan Clustering

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
from sklearn.neighbors import NearestNeighbors

neighbors = NearestNeighbors(n_neighbors=5)
neighbors_fit = neighbors.fit(x_scaled)
distances, indices = neighbors_fit.kneighbors(x_scaled)

distances = np.sort(distances, axis=0)
distances = distances[:,1]

plt.plot(distances)

In [ ]:
dbscan=DBSCAN(eps=0.135,min_samples=8)
y_dbscan=dbscan.fit_predict(x_scaled)
y_dbscan

In [ ]:
accuracy_score(y,y_dbscan)

In [ ]:
n_clusters = len(set(y_dbscan)) - (1 if -1 in y_dbscan else 0)
n_noise = list(y_dbscan).count(-1)

print('Estimated number of clusters: %d' % n_clusters)
print('Estimated number of noise points: %d' % n_noise)

In [ ]:
colors = ['mediumorchid', 'maroon', 'forestgreen','navy','goldenrod']
vectorizer = np.vectorize(lambda x: colors[x % len(colors)])

plt.scatter(x_scaled[:,0], x_scaled[:,1], c=vectorizer(y_dbscan))

Unlike k-means, DBSCAN will figure out the number of clusters. 
DBSCAN works by determining whether the minimum number of points are close enough to 
one another to be considered part of a single cluster. DBSCAN is very sensitive to scale since 
epsilon is a fixed value for the maximum distance between two points.

# Optics Clustering

# Gaussian mixtures Clustering

# Affinity propagation

In [ ]:
# from sklearn.cluster import AffinityPropagation

# afp = AffinityPropagation(damping=0.9, max_iter=200, convergence_iter=15, copy=True, preference=-5, affinity='euclidean', verbose=False, random_state=None)
# y_afp=afp.fit_predict(x_scaled)
# y_afp

# Mean-shift 

# Spectral Clustering

# Ward hierarchical